<pre>
5-point-stencil Poisson Solver:
needs grid
needs boundary values or periodicity
needs inhomogeneity values

construct matrix
construct resultant vector
solve linear system
</pre>

<pre>
try object oriented approach:
class Grid
class Stencil
...
</pre>

In [3]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#### Poisson equation on square with uniform grid and periodic boundary values

In [ ]:
def solvePoisson(f, g, a, b, N):
    '''
    Solves -Laplace[u(x,y)] = f(x,y) with the boundary condition u(x,y) = g(x,y)
    on an equidistantly discretized square [a,b] x [a,b].
    
    Returns array of function values on the grid.
    
    f ... inhomogeneity function
    g ... boundary condition function
    a ... (a, a) is lower left corner of the square
    b ... (b, b) is upper right corner of the square
    N ... number of discretization steps
    '''
    
    h = (b - a) / N
    axis = np.linspace(a, b, N+1)
    
    # generate f values on the inner grid
    y = f(axis[1:-1, None], axis[None, 1:-1])
    
    # generate boundary values
    g_bot = g(axis, a)
    g_top = g(axis, b)
    g_l = g(a, axis)
    g_r = g(b, axis)
    
    # add boundary values to sides
    y[0] += g_bot[1:-1] / h**2
    y[-1] += g_top[1:-1] / h**2
    y[:, 0] += g_l[1:-1] / h**2
    y[:, -1] += g_r[1:-1] / h**2
    
    # system matrix generation
    M = (N-1)**2
    I1 = -sp.eye(M, M, N-1)
    I2 = -sp.eye(M, M, 1-N)
    T = -sp.eye(N-1, N-1, -1) + 4*sp.eye(N-1, N-1) - sp.eye(N-1, N-1, 1)
    A = sp.block_diag((N-1)*(T,))
    L = (I1 + A + I2) / h**2
    
    # add solution of the system to output array
    u = np.zeros((N+1, N+1))
    u[1:N, 1:N] = spsolve(L, y.flatten()).reshape((N-1, N-1))
    
    # add back boundary conditions
    u[0] = g_bot
    u[-1] = g_top
    u[:, 0] = g_l
    u[:, -1] = g_r
    
    return u

Refactor: (check if sign is right)

In [9]:
def createSystemMatrix(N):
    M = (N-1)**2
    I1 = sp.eye(M, M, N-1)
    I2 = sp.eye(M, M, 1-N)
    T = sp.eye(N-1, N-1, -1) - 4*sp.eye(N-1, N-1) + sp.eye(N-1, N-1, 1)
    D = sp.block_diag((N-1)*(T,))
    return I1 + D + I2

In [8]:
def createResultVector(fVals, boundaryVals):
    ...

# Example

Laplace Equation $\Delta u = 0$
- on square $[0,1]^2$
- uniform grid with $N+1$ nodes in each direction
- step size $h = \frac{1}{N}$

Boundary values
- $u(x,0) = 0$
- $u(x,1) = \frac{1}{(1+x)^2 + 1}$
- $u(0,y) = \frac{y}{1+y^2}$
- $u(1,y) = \frac{y}{4 + y^2}$

This has exact solution $u(x,y) = \frac{y}{(1+x)^2 + y^2}$

In [17]:
N = 10
a = 0
b = 1

def BoundaryTop(x):
    return 1/((1+x)**2 + 1)

def BoundaryBottom(x):
    return 0

def BoundaryLeft(y):
    return y / (1 + y**2)

def BoundaryRight(y):
    return y / (4 + y**2)

In [14]:
def BoundaryVector(g, N):
    line = np.linspace(a, b, N+1)[1:-1]
    return np.array([g(x) for x in line])

In [28]:
def ResultMatrix(f, N):
    ...

In [ ]:
def solve(ResultMatrix, BoundaryVectors):
    N = len(BoundaryVectors[0]) + 1
    
    # subtract boundary values from the resulting values
    ResultMatrix[0] -= BoundaryVectors[0]
    ResultMatrix[-1] -= BoundaryVectors[1]
    ResultMatrix.T[0] -= BoundaryVectors[2]
    ResultMatrix.T[-1] -= BoundaryVectors[3]
    
    A = createSystemMatrix(N)
    
    spsolve(A, ResultMatrix.flatten()).reshape(N-1, N-1)